# Data Access with the Gen-2 Butler

<br>Owner: **Daniel Perrefort** ([@djperrefort](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@djperrefort))
<br>Last Verified to Run: **2019-08-08**
<br>Verified Stack Release: **v18.1.0**

## Core Concepts

This notebook provides a hands on overview of how to interact with the Gen-2 `Butler` (it should be updated for Gen-3, once available). The `Butler` provides a way to access information using a uniform interface without needing to keep track of how the information is internally stored or organized. Data access with `Butler` has three levels you need to be aware of:

1. Each instantiated `Butler` object provides access to a collection of datasets called a **repository**. Each repository is defined by Butler using the local file directory where the data is stored.
1. Each data set in a **repository** is assigned a unique name called a **type**. These types are strings that describe the data set and should not be confused with an "object type" as defined by Python.
1. Individual entries in a data set are identified using a unique **data identifier**, which is a dictionary who's allowed keys and values depend on the data set you are working with.

## Learning Objectives:

This notebook demonstrates how to use the Gen-2 `Butler` object from the DM stack to parse and manipulate data. After finishing this notebook, users will know how to:

- Load and access a data repository using `Butler`
- Select subsets of data and convert data into familiar data structures
- Use `Butler` to access coordinate information and cutout postage stamps
- Use `Butler` to access a skymap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
import matplotlib.pyplot as plt
from lsst.daf.persistence import Butler
from lsst.geom import SpherePoint

%matplotlib inline

## Loading Data

To start we instantialize a `Butler` object by providing it with the directory of the **repository** we want to access. Next we load a **type** of dataset and select data from a single **data identifier**. For this demonstration we consider the `deepCoadd_ref` dataset which contains tables of information concerning coadded images used in the differencing image pipeline. The id values for this data set include two required values: `tract` and `patch` which denote sections of the sky.


In [ ]:
#repo = '/project/shared/data/diffim_template'
#repo = '/project/shared/data/DATA_ci_hsc/rerun/coadd'
repo = '/project/shared/data/DATA_ci_hsc/rerun/coaddForcedPhot'
butler = Butler(repo)


In [ ]:
butler.get('deepCoadd_ref', {'tract': 0, 'patch': '1,1'})


In [ ]:
# We choose an arbitrary tract and patch. 
data_id = {'tract': 0, 'patch': '1,1'}
dataset_type = 'deepCoadd_ref'

# We can check that the data exists before we try to read it
data_exists = butler.datasetExists(datasetType=dataset_type, dataId=data_id)
print('Data exists for ID:', data_exists)

data_entry = butler.get(dataset_type, dataId=data_id)
data_entry


The data table returned above is formatted as a `SourceCatalog` object, which is essentially a collection of `numpy` arrays. We can see this when we index a particular column. 

In [ ]:
print(type(data_entry['merge_measurement_i']))


`SourceCatalog` objects have their own set of methods for table manipulations (sorting, appending rows, et.). However, we can also work with the data in a more familiar format, such as an astropy `Table` or a pandas `DataFrame`. 

In [ ]:
data_frame = data_entry.asAstropy().to_pandas()
data_frame.head()


It is important to note that `Butler` objects are not required to return tabular data. We will see an example of this later when we load and parse image data.

## Selecting Subsets of Data

In practice you may not know the format of the data identifier for a given data set. In this case the `getKeys()` method can be used to determine the key values expected in a **data identifier**. 


In [ ]:
data_id_format = butler.getKeys(dataset_type)
print('Expected data id format:', data_id_format)


It is import to note that the `getKeys` method will only specify key values that are required by the data set. We can also include optional key values to further select a subset of the data. Lets say we only wanted table entries where `merge_footprint_i` has a value of `True`. We could retrieve the data and subselect the values of the returned table, or we can do simply specify the condition in the data id.

In [ ]:
# Don't do this!
#new_data_id = {'tract': 0, 'patch': '1,1', 'merge_footprint_i': True}
# Do this...
new_data_id = {'tract': 0, 'patch': '1,1'}
merged_i_data = butler.get(dataset_type, dataId=new_data_id)

# Check that the returned table does in fact have only entries where
# merge_footprint_i is True
# Need to copy or else non-contiguous in memory (yay)
merged_i_data = merged_i_data[merged_i_data['merge_measurement_i']].copy(True)

print(merged_i_data['merge_measurement_i'].all())
merged_i_data.asAstropy().to_pandas()

In [ ]:
np.where( (merged_i_data['base_PsfFlux_flux'] > 100) & (merged_i_data['base_PsfFlux_flux'] < 500))

In [ ]:

i = 1000
ra =  np.degrees(merged_i_data['coord_ra'][i])
dec = np.degrees(merged_i_data['coord_dec'][i])

Although you can't specify complex conditionals in a data id, you can use it so perform simple selections. You can also can also select a select all complete dataIds for a dataset type that match a partial (or empty) dataId. For example, the below cell iterates over all possible ids and checks if the corresponding file exists.


In [ ]:
subset = butler.subset(dataset_type, dataId=data_id)
id_list = [dr.dataId for dr in subset if dr.datasetExists()]
print(f'fAvailable Ids:\n {id_list}')


## Creating Postage Stamps

When dealing with image data, we can use `Butler` to generate postage stamps at a given set of coordinates. For this example we consider the `deepCoadd` data set, which has one extra key value than the previous example.

In [ ]:
coadd_type = 'deepCoadd'
butler.getKeys(coadd_type)


In order to generate a postage stamp, we define the center an size of the cutout using 

In [ ]:
# Retrieve the image using butler
coadd_id = {'tract': 0, 'patch': '1,1', 'filter': 'HSC-I'}
image = butler.get(coadd_type, dataId=coadd_id)

# Define the center and size of our cutout
#ra, dec = 53.135801, -28.426165
#ra, dec = 320.97720068290386, -0.4069160376552347
radec = SpherePoint(ra, dec, afwGeom.degrees)
cutout_size = 150
cutout_extent = afwGeom.ExtentI(cutout_size, cutout_size)

# Cutout and optionally save the postage stamp to file
postage_stamp = image.getCutout(radec, cutout_extent)
# postage_stamp.writeFits(<output_filename>)


Since the postage stamp was generated using `Butler`, we can plot it using the DM `afwDisplay` module.

In [ ]:
xy = postage_stamp.getWcs().skyToPixel(radec)

display = afwDisplay.Display(frame=1, backend='matplotlib')
display.mtv(postage_stamp)
display.scale("linear", "zscale")
display.dot('o', xy.getX(), xy.getY(), ctype='red')
display.show_colorbar()

plt.xlabel('x')
plt.ylabel('y')
plt.show()


It is important to note that the cutout image is aware of what the pixel values were in the original image. This is why the axis labels in the above cutout are so large. We also note that the orientation of the postage stamp is in the x, y orientation of the original coadded image.


## Selecting an Area on the Sky with a Sky Map

As a final example, we consider a third type of data that can be accessed via `Butler` called `skymap`. Sky maps allow you to look up information for a given `tract` and `patch`. You may notice from the below example that data set **types** tend to follow the convertion of having a base name (e.g. `'deepCoadd'`) followed by a descriptor (e.g. `'_skyMap'`).

In [ ]:
skymap = butler.get('deepCoadd_skyMap')
tract_info = skymap[0]
tract_info


In [ ]:
patch_info = tract_info.getPatchInfo((1,1))
patch_info


In [ ]:
tract_bbox = tract_info.getBBox()
tract_pix_corners = afwGeom.Box2D(tract_bbox).getCorners()
print('Tract corners in pixels:\n', tract_pix_corners)

wcs = tract_info.getWcs()
tract_deg_corners = wcs.pixelToSky(tract_pix_corners)
tract_deg_corners = [[c.getRa().asDegrees(), c.getDec().asDegrees()] for c in tract_deg_corners]
print('\nTract corners in degrees:\n', tract_deg_corners)
